In [1]:
import os

In [9]:
baudrillard = '/Users/finchmf/coding/TwitterRitual/core/behavior/text/webscraper/baudrillard/textData/simulacra-and-simulation.txt'
baudrillard_2 = '/Users/finchmf/coding/TwitterRitual/core/behavior/text/webscraper/baudrillard/textData/simulations.txt'
def load_data(path: str) -> str:

    text_file = os.path.join(path)
    with open(text_file, "r") as f:
        data = f.read()

    return data

In [10]:
data = load_data(baudrillard)
data_2 = load_data(baudrillard_2)

In [13]:
main_data = data + data_2

In [17]:
import numpy as np

view_line_range = (100, 150)

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in main_data.split()})))

lines = main_data.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(main_data.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 14440
Number of lines: 11204
Average number of words in each line: 7.909139593002499

The lines 100 to 150:
whose operation is nuclear and genetic, no longer at all specular or discursive. It is all of 
metaphysics that is lost. No more mirror of being and appearances, of the real and its 
concept. No more imaginary coextensivity: it is genetic miniaturization that is the 
dimension of simulation. The real is produced from miniaturized cells, matrices, and 
memory banks, models of control - and it can be reproduced an indefinite number of 
times from these. It no longer needs to be rational, because it no longer measures itself 
against either an ideal or negative instance. It is no longer anything but operational. In 
fact, it is no longer really the real, because no imaginary envelops it anymore. It is a 
hyperreal, produced from a radiating synthesis of combinatory models in a hyperspace 
without atmosphere. 

By crossing into a spac

In [ ]:
from collections import Counter

def create_lookup_tables(text: str) -> tuple:

    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    i2w = {idx: word for idx, word, in enumerate(sorted_vocab)}
    w2i = {word: idx for idx, word in i2w.items()}

    return (w2i, i2w)

def token_lookup():
    return {
        '.': '||period||',
        ',': '||comma||',
        '"': '||quotation_mark||',
        ';': '||semicolon||',
        '!': '||exclamation_mark||',
        '?': '||question_mark||',
        '(': '||left_parentheses||',
        ')': '||right_Parentheses||',
        '-': '||dash||',
        '\n': '||return||'
    }

